## Einrichtung von Julia JuMP
---
Überprüfen Sie die bereits installierten Pakete. In der JuliaBox sind möglicherweise alle notwendigen Pakete bereits vorhanden.

In [233]:
using Pkg;
Pkg.status()

    Status `~/.julia/environments/v1.2/Project.toml`
  [336ed68f] CSV v0.5.16
  [a93c6f00] DataFrames v0.19.4
  [60bf3e95] GLPK v0.11.4
  [3c7084bd] GLPKMathProgInterface v0.4.4
  [7073ff75] IJulia v1.20.0
  [4076af6c] JuMP v0.20.0
  [f3b207a7] StatsPlots v0.12.0


Installieren Sie ``JuMP`` , mit dem Sie einfach Optimierungsprogramme formulieren können. Sowie ``GLPK`` einen open-source Solver zum Lösen der Problem und ``StatsPlots`` zur Visualisierung der Lösung.

In [234]:
#Pkg.add("JuMP")
#Pkg.add("GLPK");
#Pkg.add("StatsPlots");

Richten Sie die installierten Pakete so ein, dass sie im folgenden Code verwendet werden können.

In [235]:
using JuMP, GLPK, StatsPlots;

# Planspiel Schritt 2: Capacitated Lot Sizing Problem



### Ergebnisse aus Teil 1

In [236]:
F_Aggr = [0.0  0.0  0.0  0.0  0.0  0.0
         0.0  0.0  0.0  0.0  0.0  0.0
         0.0  0.0  0.0  0.0  0.0  0.0];

In [237]:
ZAggr = [21.0  6.0  0.0  0.0  38.0  40.0];

In [238]:
LAggr = [0.0  0.0  0.0  823.0  902.0  647.0  0.0
         0.0  0.0  0.0    0.0    1.0    0.0  0.0
         0.0  0.0  0.0    9.0    3.0    1.0  0.0 ];

## Erstellen Sie das Modell namens ``m`` und geben Sie als zu verwendenden Solver GLPK an.
---

In [239]:
m = Model(with_optimizer(GLPK.Optimizer, tm_lim=60000));

# tm_lim beschränkt die Rechenzeit für die Optimierung auf 60 Sekunden

### Mengen und Parameter

Fügen Sie die Mengen ein.


In [240]:
#Mengen
Produkte =   [];
Perioden =   []; 
# Hinweis: Erstellen Sie für jeden Monat ein eigenes Modell (mit je 4 Wochen)

Ressourcen = [];


#Längen
I = length(Produkte);
T = length(Perioden);
J = length(Ressourcen);

Fügen Sie die Parameter ein.

In [241]:
#Hinweis: Überlegen Sie welche Parameter von Monats- auf Wochenwerte heruntergerechnet werden müssen

kl = [];                        #Lagerkostensatz der Produkte pro Woche

ks = [];                        #Rüstkostensatz der Produkte

ts = [];                        #Rüstzeiten der Produkte

tp = [0.105, 0.095, 0.065];     #Stückbearbeitungszeiten der Produkte

M = 100000;                     #Große Zahl

                   
d = [];  
#Bedarf von Produkt i in Periode t

kf = [320, 432, 240];        #Fremdbezugskostensatz 1 der Produkte (Tucher)

kv = [160, 216, 120];          #variable Herstellkosten der Produkte

In [242]:
#Diese Werte sind abhängig von der aggregierten Planung

l_start = [];              #Anfangslagerbestand des Monats (Endbestand Vorperiode, s. oben)

l_end = [];                #Der Endbestand des Monats ist immer der entsprechend berechnete Lagerbestand aus Schritt 1

c =  [];                        #Periodenkapazität
#Hinweis: Berücksichtigen Sie zusätzlich zur regulären Kapazität auch die in der aggr. Planung 
#         ermittelte Zusatzkapazität aufgeteilt auf 4 Wochen

faggr = []                 #Die wöchentliche Lieferung der bei Cheapress bestellten Produkte

### Entscheidungsvariablen
---
Definieren Sie die Entscheidungsvariablen. Achten Sie auf die Definitionsbereiche.

**Nichtnegativitätsbedingung**: Die Entscheidungsvariablen, also die Produktionsmengen, Lagerbestände, Fremdbezugsmengen und die in Anspruch genommenen Zusatzkapazitäten dürfen nur positive Werte annehmen, wir setzen deren Nichtnegativität voraus.

$ \qquad X_{it}, L_{it}, F_{it} \geq 0 \qquad \qquad \forall i \in I, t \in T$


In [243]:
@variables  m begin
            X[1:I,1:T]>=0,Int  #Produktionsmenge von Produkt i in Periode t
            F[1:I,1:T]>=0,Int  #Fremdbezugsmenge von Produkt i in Periode t
            L[1:I,0:T]>=0, Int; #Lagerbestand von Produkt i am Periodenende von t
            end

**Binaerbedingung**: Rüstvorgänge werden immer ganz ($\gamma_{it} = 1$) oder gar nicht ($\gamma_{it} = 0$) ausgeführt. Die binäre Rüstvariable nimmt also entweder den Wert 1 oder 0 an.

 $ \qquad  \gamma_{it} \in \{0,1\}  \qquad\qquad\qquad\qquad\qquad\qquad \forall t \in T, \quad i \in I $

In [244]:
@variable(m,gamma[1:I,1:T],Bin); #Die binäre Rüstvariable von Produkt i in Periode t

## Zielfunktion

**Kostenminimierung:** Der Produktionskosten K sollen minimiert werden. Diese berechnen sich hier aus der Summe der variablen Kosten, der Lagerkosten, der Fremdbezugskosten und Rüstkosten.

$$  \qquad \min K = \displaystyle\sum_{i=1}^I \sum_{t=1}^T (k_i^vX_{it}+k_i^l L_{it}+k_i^fF_{it}+ k^s_i\gamma_{i,t}) $$

In [1]:
@objective(m, Min,  sum(kv[i] * X[i,t] + kl[i] * L[i,t] + kf[i] * F[i,t] + ks[i]*gamma[i,t]
                    for i=1:I for t=1:T));

LoadError: UndefVarError: @objective not defined

## Nebenbedingungen

**Kapazitaetsrestriktion:** Die Summe aus Produktions- und Rüstzeit aller Produkte darf in jeder Periode die vorhandene Kapazität der Ressource j nicht überschreiten.

 $$ \qquad \displaystyle\sum_{i=1}^I(t_i^p\cdot X_{it}+t_i^s\cdot\gamma_{it} ) \leq c_t \hspace{40mm} \forall t \in T, j \in J\qquad  $$ 
 


In [246]:
@constraint(m, KapRes[t=1:T, j=1:J], sum(ts[i] * gamma[i,t] + tp[i] * X[i,t] for i=1:I) <= c[t] );

**Lagerbilanzgleichung**: Der Lagerbestand eines Produktes am Ende einer Periode berechnet sich aus der Summe der eingelagerten Menge in der Vorperiode, der Produktionsmenge und der kurzfristigen und langfristigen Fremdbezugsmenge, abzüglich der abgesetzen Menge des Produktes.

$$ \qquad L_{it}=L_{i,t-1}+X_{it}-d_{it}+F_{it}+f_{it}^{aggr} \qquad \forall i \in I , t \in T$$

In [247]:
#Fügen sie den Parameter der langfristigen Fremdbezugsmenge in die Nebenbedingung ein!

@constraint(m, Lager[i=1:I,t=1:T], L[i,t] == L[i,t-1] + X[i,t] + F[i,t] - d[i,t]);

**Anfangslagerbestand**: Der Anfangslagerbestand aller Produkte entspricht dem initial gesetzen $l_{start}$.

**Endlagerbestand**: Der Endlagerbestand aller Produkte entspricht dem initial gesetzen $l_{end}$.
 
  $$ \qquad L_{i,0} = l_{start} \hspace{40mm} \forall  i \in I$$
  
  $$ \qquad L_{i,T} = l_{end} \hspace{40mm} \forall  i \in I$$


In [248]:
@constraint(m, AnfLager[i=1:I], L[i,0] == l_start[i]);

#Fügen Sie eine Nebenbedingung für den Endlagerbestand hinzu

@contraint()

**Rüstbedingung**: Wenn für ein Produkt in einer Periode nicht gerüstet wird, ist die produzierte Menge dieses Produkts in dieser Periode 0. Wenn für ein Produkt in einer Periode gerüstet wird, wird die produzierte Menge durch die Rüstbedingung nicht eingeschränkt.

 $$ \qquad X_{it} \leq M \cdot \gamma_{it} \hspace{40mm} \forall t \in T, \quad i \in I $$

In [249]:
@constraint(m, Ruestbed[i=1:I,t=1:T], X[i,t] <= M * gamma[i,t]);

## Lösen Sie das Modell.
---

In [253]:
#Füge den fehlenden Befehl ein

Lassen Sie sich den Zielfunktionswert Z anzeigen.

In [254]:
cost = #Befehl zum Anzeigen des Zielfunktionswertes einfügen

println("Objective value cost: ", round(Int64,db))

Objective value Z: 1.1626662025e6


Platz für weitere Berechnungen: